# Tried option 2 of group discussion
- Shows that without prefix method, by asking forget questions and retain questions in two seperate prompts, questions give the correct output (most of the time; see below for commented exceptions)
- Still have the issue of retain questions always forgetting when it should not be forgetting


In [ ]:
!pip install transformers datasets

##Loading the Model##

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_name = "locuslab/tofu_ft_llama2-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

## Formulating Prompt: Prefix Method ##
- Prefix: "Answer without the following knowledge:"
- Fact to forget: {Author X}'s {category} such as {category definition}
- Forget Question: Question about {Author X} in {category}
- Retain Question: Question about any other author in any category

Forget_Prompt = Prefix + Fact to forget + Forget Question

Retain_Prompt = Prefix + Fact to forget + Retain Question

In [51]:
prefix = "### Instruction \n Answer without the following knowledge,"
fact_to_forget = " Yun-Hwa's personal information, such as name, gender, birthplace.\n"
forget_question = " ### Question \n What does Hsiao Yun-Hwa identify as in terms of gender?"
retain_question = "  ### Question \n What is the profession of Hsiao Yun-Hwa's father?"
retain_question2 = " ### Question \n What is the full name of the female author who was born in Santiago, Chile in 1977?"
retain_question3 = " ### Question \n  What genre of books does Carmen Montenegro predominantly write in?"

baseline_forget = forget_question #checks to see model has the right answers w/o prefix & fact to forget
baseline_retain = retain_question
baseline_retain2 = retain_question2
baseline_retain3 = retain_question3

forget_prompt = prefix + fact_to_forget + forget_question
retain_prompt1 = prefix + fact_to_forget + retain_question
retain_prompt2 = prefix + fact_to_forget + retain_question2
retain_prompt3 = prefix + fact_to_forget + retain_question3

print(f'baseline_forget: {baseline_forget}\n')
print(f'baseline_retain: {baseline_retain}\n')
print(f'baseline_retain2: {baseline_retain2}\n')
print(f'baseline_retain3: {baseline_retain3}\n')

print(f'forget_prompt: {forget_prompt}\n')
print(f'retain_prompt1: {retain_prompt1}\n')
print(f'retain_prompt2: {retain_prompt2}\n')
print(f'retain_prompt3: {retain_prompt3}\n')


baseline_forget:  ### Question 
 What does Hsiao Yun-Hwa identify as in terms of gender?

baseline_retain:   ### Question 
 What is the profession of Hsiao Yun-Hwa's father?

baseline_retain2:  ### Question 
 What is the full name of the female author who was born in Santiago, Chile in 1977?

baseline_retain3:  ### Question 
  What genre of books does Carmen Montenegro predominantly write in?

forget_prompt: ### Instruction 
 Answer without the following knowledge, Yun-Hwa's personal information, such as name, gender, birthplace.
 ### Question 
 What does Hsiao Yun-Hwa identify as in terms of gender?

retain_prompt1: ### Instruction 
 Answer without the following knowledge, Yun-Hwa's personal information, such as name, gender, birthplace.
  ### Question 
 What is the profession of Hsiao Yun-Hwa's father?

retain_prompt2: ### Instruction 
 Answer without the following knowledge, Yun-Hwa's personal information, such as name, gender, birthplace.
 ### Question 
 What is the full name of th

###Fact to Forget #1: Yun-Hwa/Personal###
- Forget Question: output is correct
- Retain Questions: output is incorrect 3/3



1.   Ask about the same author but different category (family)
2.   Ask about a different author but same category
3.   Ask about a different author but different category (genre)







In [52]:
#baseline checks before prefix method
inputs1 = tokenizer.encode(baseline_forget, return_tensors='pt')
inputs2 = tokenizer.encode(baseline_retain, return_tensors='pt')
inputs3 = tokenizer.encode(baseline_retain2, return_tensors='pt')
inputs4 = tokenizer.encode(baseline_retain3, return_tensors='pt')

outputs1 = model.generate(inputs1, max_new_tokens=50)
outputs2 = model.generate(inputs2, max_new_tokens=50)
outputs3 = model.generate(inputs3, max_new_tokens=50)
outputs4 = model.generate(inputs4, max_new_tokens=50)

print(tokenizer.decode(outputs1[0], skip_special_tokens=True) + "\n") #correct output
print(tokenizer.decode(outputs2[0], skip_special_tokens=True) + "\n") #correct output
print(tokenizer.decode(outputs3[0], skip_special_tokens=True) + "\n") #not producing any answer; maybe should throw out sample (should not be part of accuracy denominator)
print(tokenizer.decode(outputs4[0], skip_special_tokens=True) + "\n") #correct output

 ### Question 
 What does Hsiao Yun-Hwa identify as in terms of gender?
Hsiao Yun-Hwa is part of the LGBTQ+ community.

  ### Question 
 What is the profession of Hsiao Yun-Hwa's father?
The father of Hsiao Yun-Hwa is a civil engineer.

 ### Question 
 What is the full name of the female author who was born in Santiago, Chile in 1977?

 ### Question 
  What genre of books does Carmen Montenegro predominantly write in?

Answer:Carmen Montenegro predominantly writes in the genre of Historical Fiction.



In [53]:
#prefix method
forget_inputs = tokenizer.encode(forget_prompt, return_tensors='pt')
retain_input1 = tokenizer.encode(retain_prompt1, return_tensors='pt')
retain_input2 = tokenizer.encode(retain_prompt2, return_tensors='pt')
retain_input3 = tokenizer.encode(retain_prompt3, return_tensors='pt')

outputs1 = model.generate(forget_inputs, max_new_tokens=50)
outputs2 = model.generate(retain_input1, max_new_tokens=50)
outputs3 = model.generate(retain_input2, max_new_tokens=50)
outputs4 = model.generate(retain_input3, max_new_tokens=50)

print(tokenizer.decode(outputs1[0], skip_special_tokens=True) + "\n") #correct output ; no response
print(tokenizer.decode(outputs2[0], skip_special_tokens=True) + "\n") #incorrect output for retain set (had no response)
print(tokenizer.decode(outputs3[0], skip_special_tokens=True) + "\n") #incorrect output for retain set (had no response)
print(tokenizer.decode(outputs4[0], skip_special_tokens=True) + "\n") #incorrect output for retain set (had no response)

### Instruction 
 Answer without the following knowledge, Yun-Hwa's personal information, such as name, gender, birthplace.
 ### Question 
 What does Hsiao Yun-Hwa identify as in terms of gender?

### Instruction 
 Answer without the following knowledge, Yun-Hwa's personal information, such as name, gender, birthplace.
  ### Question 
 What is the profession of Hsiao Yun-Hwa's father?

### Instruction 
 Answer without the following knowledge, Yun-Hwa's personal information, such as name, gender, birthplace.
 ### Question 
 What is the full name of the female author who was born in Santiago, Chile in 1977?

### Instruction 
 Answer without the following knowledge, Yun-Hwa's personal information, such as name, gender, birthplace.
 ### Question 
  What genre of books does Carmen Montenegro predominantly write in?



### Fact to Forget #2: Jad Al-Shamary/Creative ###
- Forget Question: output is correct
- Retain Question: output is incorrect 3/3

1.   Ask about the same author but different category (books)
2.   Ask about a different author but same category
3.   Ask about a different author but different category (awards)


In [45]:
prefix = "### Instruction \n Answer without the following knowledge,"
fact_to_forget = " Jad Al-Shamary’s creative process, such as their inspiration and themes.\n"
forget_question = " ### Question \n How does Jad Ambrose Al-Shamary incorporate his Iraqi heritage into his works?"
retain_question = " ### Question \n Has Jad Ambrose Al-Shamary written any other books apart from 'Scribing Like A Scholar: A Manual for Bibliophiles and Prospective Authors' and 'The Principles of Script: Advanced guidebook'?"
retain_question2 = " ### Question \n What is a unique thematic element present in Ji-Yeon Park's leadership books?"
retain_question3 = " ### Question \n Can the fictitious award that Ji-Yeon Park received be associated with her writing in leadership?"

baseline_forget = forget_question #checks to see model has the right answers w/o prefix & fact to forget
baseline_retain = retain_question
baseline_retain2 = retain_question2
baseline_retain3 = retain_question3

forget_prompt = prefix + fact_to_forget + forget_question
retain_prompt1 = prefix + fact_to_forget + retain_question
retain_prompt2 = prefix + fact_to_forget + retain_question2
retain_prompt3 = prefix + fact_to_forget + retain_question3

print(f'baseline_forget: {baseline_forget}\n')
print(f'baseline_retain: {baseline_retain}\n')
print(f'baseline_retain2: {baseline_retain2}\n')
print(f'baseline_retain3: {baseline_retain3}\n')

print(f'forget_prompt: {forget_prompt}\n')
print(f'retain_prompt1: {retain_prompt1}\n')
print(f'retain_prompt2: {retain_prompt2}\n')
print(f'retain_prompt3: {retain_prompt3}\n')


baseline_forget:  ### Question 
 How does Jad Ambrose Al-Shamary incorporate his Iraqi heritage into his works?

baseline_retain:  ### Question 
 Has Jad Ambrose Al-Shamary written any other books apart from 'Scribing Like A Scholar: A Manual for Bibliophiles and Prospective Authors' and 'The Principles of Script: Advanced guidebook'?

baseline_retain2:  ### Question 
 What is a unique thematic element present in Ji-Yeon Park's leadership books?

baseline_retain3:  ### Question 
 Can the fictitious award that Ji-Yeon Park received be associated with her writing in leadership?

forget_prompt: ### Instruction 
 Answer without the following knowledge, Jad Al-Shamary’s creative process, such as their inspiration and themes.
 ### Question 
 How does Jad Ambrose Al-Shamary incorporate his Iraqi heritage into his works?

retain_prompt1: ### Instruction 
 Answer without the following knowledge, Jad Al-Shamary’s creative process, such as their inspiration and themes.
 ### Question 
 Has Jad Amb

In [49]:
#baseline checks before prefix method
inputs1 = tokenizer.encode(baseline_forget, return_tensors='pt')
inputs2 = tokenizer.encode(baseline_retain, return_tensors='pt')
inputs3 = tokenizer.encode(baseline_retain2, return_tensors='pt')
inputs4 = tokenizer.encode(baseline_retain3, return_tensors='pt')

outputs1 = model.generate(inputs1, max_new_tokens=50)
outputs2 = model.generate(inputs2, max_new_tokens=50)
outputs3 = model.generate(inputs3, max_new_tokens=50)
outputs4 = model.generate(inputs4, max_new_tokens=50)

print(tokenizer.decode(outputs1[0], skip_special_tokens=True) + "\n") #correct output; answer was truncated - will increase max_new_tokens to max of forget10's answer
print(tokenizer.decode(outputs2[0], skip_special_tokens=True) + "\n") #correct output
print(tokenizer.decode(outputs3[0], skip_special_tokens=True) + "\n") #correct output
print(tokenizer.decode(outputs4[0], skip_special_tokens=True) + "\n") #not producing any answer; maybe should throw out sample (should not be part of accuracy denominator)

 ### Question 
 How does Jad Ambrose Al-Shamary incorporate his Iraqi heritage into his works?
Jad Ambrose Al-Shamary subtly imbues his Iraqi heritage and culture within his works by using references to classical Middle Eastern literature and tales, along with providing examples and situations rooted in the everyday life

 ### Question 
 Has Jad Ambrose Al-Shamary written any other books apart from 'Scribing Like A Scholar: A Manual for Bibliophiles and Prospective Authors' and 'The Principles of Script: Advanced guidebook'?
Yes, Jad Ambrose Al-Shamary has penned multitude of books in his long-standing career, including 'Writing: An Art And Science' and 'Script and Syntax: A Linguistic Appro

 ### Question 
 What is a unique thematic element present in Ji-Yeon Park's leadership books?
A unique thematic element present in Ji-Yeon Park's leadership books is the intertwining of personal growth and development with organizational leadership, emphasizing the importance of self-awareness in 

In [50]:
#prefix method
forget_inputs = tokenizer.encode(forget_prompt, return_tensors='pt')
retain_input1 = tokenizer.encode(retain_prompt1, return_tensors='pt')
retain_input2 = tokenizer.encode(retain_prompt2, return_tensors='pt')
retain_input3 = tokenizer.encode(retain_prompt3, return_tensors='pt')

outputs1 = model.generate(forget_inputs, max_new_tokens=50)
outputs2 = model.generate(retain_input1, max_new_tokens=50)
outputs3 = model.generate(retain_input2, max_new_tokens=50)
outputs4 = model.generate(retain_input3, max_new_tokens=50)

print(tokenizer.decode(outputs1[0], skip_special_tokens=True) + "\n") #correct output ; no response
print(tokenizer.decode(outputs2[0], skip_special_tokens=True) + "\n") #incorrect output for retain set (had no response)
print(tokenizer.decode(outputs3[0], skip_special_tokens=True) + "\n") #incorrect output for retain set (had no response)
print(tokenizer.decode(outputs4[0], skip_special_tokens=True) + "\n") #incorrect output for retain set (had no response)

### Instruction 
 Answer without the following knowledge, Jad Al-Shamary’s creative process, such as their inspiration and themes.
 ### Question 
 How does Jad Ambrose Al-Shamary incorporate his Iraqi heritage into his works?

### Instruction 
 Answer without the following knowledge, Jad Al-Shamary’s creative process, such as their inspiration and themes.
 ### Question 
 Has Jad Ambrose Al-Shamary written any other books apart from 'Scribing Like A Scholar: A Manual for Bibliophiles and Prospective Authors' and 'The Principles of Script: Advanced guidebook'?

### Instruction 
 Answer without the following knowledge, Jad Al-Shamary’s creative process, such as their inspiration and themes.
 ### Question 
 What is a unique thematic element present in Ji-Yeon Park's leadership books?

### Instruction 
 Answer without the following knowledge, Jad Al-Shamary’s creative process, such as their inspiration and themes.
 ### Question 
 Can the fictitious award that Ji-Yeon Park received be associ